In [1]:
import pandas as pd
import collections

import numpy as np
import folium as fo
import json
from bokeh.io import show,  output_notebook, export_png
from bokeh.plotting import figure
from bokeh.models import LabelSet,Legend
output_notebook()
%matplotlib inline

Loading BokehJS ...

In [2]:
df=pd.read_parquet('resources/cleaned_franchises.parquet')
df['Results'] = (df['Results'] == 'Fail')
df = df.rename(columns={'Results': 'Fails'})
df['Inspection Date'] = df['Inspection Date'].apply(lambda x: x.year)
df = df[df['Inspection Date'] !=2018]
df = df[df['Inspection Date']!=2019]
df.head()

,DBA Name,AKA Name,License #,Facility Type,Risk,Address,Zip,Inspection Date,Inspection Type,Fails,...,Re-inspection,Community,Neighborhood,Ward,Precinct,Violation IDs,Nb Violations,Nb Violations of high severity,Nb Violations of medium severity,Nb Violations of low severity
Inspection ID,,,,,,,,,,,,,,,,,,,,,
2129810,PICCOLO MONDO CAFE,PICCOLOMONDOCAFE,1908007,Restaurant,High,1642 E 56TH ST,60637,2017,Suspect food poisoning,True,...,False,HYDE PARK,HYDE PARK,5,35,"[3, 18, 34, 35, 38]",5,1,1,3
2129821,STARBUCKS COFFEE # 216,STARBUCKS,2398564,Restaurant,Medium,401 E ONTARIO ST,60611,2017,Canvass,False,...,False,NEAR NORTH SIDE,STREETERVILLE,42,7,[37],1,0,0,1
2129811,NORTHSIDE GRILL,NORTHSIDEGRILL,2404396,Restaurant,High,4351 N ELSTON AVE,60641,2017,Canvass,False,...,False,IRVING PARK,"IRVING PARK,AVONDALE",39,25,"[30, 32, 33, 34, 35, 38, 41]",7,0,0,7
2129808,DUNKIN DONUTS,DUNKINDONUTS,2391841,Restaurant,Medium,6701 S JEFFERY BLVD,60649,2017,Complaint,False,...,False,SOUTH SHORE,"SOUTH SHORE, GRAND CROSSING",5,1,[21],1,0,1,0
2129815,THE LUNCH ROOM,THELUNCHROOM,2559651,Restaurant,High,70 W MADISON ST,60602,2017,License,False,...,False,LOOP,LOOP,42,9,[None],0,0,0,0


In [3]:
nb_licenses_by_aka_dict = {}
for name in df['AKA Name'].unique():
    nb_uniques = len(df[df['AKA Name'] == name].groupby('License #'))
    nb_licenses_by_aka_dict[name] = nb_uniques
nb_licenses_by_aka = pd.DataFrame(nb_licenses_by_aka_dict.items(), columns=['AKA Name', 'Nb Licenses']).set_index('AKA Name').sort_values('Nb Licenses', ascending=False)
nb_licenses_by_aka.head()

,Nb Licenses
AKA Name,
SUBWAY,370
DUNKINDONUTS,248
STARBUCKS,218
MCDONALDS,157
BURGERKING,100


In [4]:
df_mc = df[df['AKA Name']== 'MCDONALDS']
df_bk = df[df['AKA Name']== 'BURGERKING']
df_kfc = df[df['AKA Name']== 'KFC']
df_hc = df[df['AKA Name']== 'HAROLDSCHICKEN']
df_sw = df[df['AKA Name']== 'SUBWAY']
df_ps = df[df['AKA Name']== 'POTBELLYSANDWICH']

In [5]:
col_to_drop=['DBA Name',
 'AKA Name',
 'Facility Type',
 'Violations',
 'License #',
 'Risk',
 'Address',
 'Zip',
 'Inspection Type',
 'Latitude',
 'Longitude',
 'Re-inspection',
 'Community',
 'Neighborhood',
 'Ward',
 'Precinct',
 'Violation IDs']
col_to_rename= {'Nb Violations of high severity': 'high', 'Nb Violations of medium severity': 'medium', 'Nb Violations of low severity': 'low'}

In [6]:
def nb_inspections (data,franchise):
    nb_inspections_by_year = data[['Inspection Date', 'License #']].copy()
    nb_inspections_by_year.rename(columns={'Inspection Date' : 'Inspection Year'}, inplace=True)
    nb_inspections_by_year = nb_inspections_by_year.groupby('Inspection Year').count().rename(columns={'License #' : 'Count'})
    nb_inspections_by_year.Count = nb_inspections_by_year.Count/nb_licenses_by_aka.loc[franchise,"Nb Licenses"]
    return nb_inspections_by_year

In [7]:
def df_violations(data):
    df_violations = data.drop(col_to_drop,axis=1).rename(columns =col_to_rename)
    df_violations ['Inspections'] = 1
    df_violations  = df_violations .set_index('Inspection Date')
    return df_violations


In [8]:
def agg_year(violations,inspections):
    agg_year = violations.groupby('Inspection Date').agg(sum)
    agg_year['high'] = agg_year.high / agg_year['Nb Violations']
    agg_year['medium'] = agg_year.medium / agg_year['Nb Violations']
    agg_year['low'] = agg_year.low / agg_year['Nb Violations']
    agg_year['inspection_fail_rate'] = agg_year.Fails / agg_year.Inspections
    agg_year['Mean inspections']=inspections.Count 
    return agg_year

In [9]:
nb_inspections_by_year_mc = nb_inspections(df_mc,'MCDONALDS')
nb_inspections_by_year_bk = nb_inspections(df_bk,'BURGERKING')
nb_inspections_by_year_kfc = nb_inspections(df_kfc,'KFC')
nb_inspections_by_year_hc = nb_inspections(df_hc,'HAROLDSCHICKEN')
nb_inspections_by_year_sw = nb_inspections(df_sw,'SUBWAY')
nb_inspections_by_year_ps = nb_inspections(df_ps,'POTBELLYSANDWICH')

In [10]:
df_mc_violations = df_violations(df_mc)
df_bk_violations = df_violations(df_bk)
df_kfc_violations = df_violations(df_kfc)
df_hc_violations = df_violations(df_hc)
df_sw_violations = df_violations(df_sw)
df_ps_violations = df_violations(df_ps)

In [11]:
agg_year_mc = agg_year(df_mc_violations,nb_inspections_by_year_mc)
agg_year_bk = agg_year(df_bk_violations,nb_inspections_by_year_bk)
agg_year_kfc = agg_year(df_kfc_violations,nb_inspections_by_year_kfc)
agg_year_hc = agg_year(df_hc_violations,nb_inspections_by_year_hc)
agg_year_sw = agg_year(df_sw_violations,nb_inspections_by_year_sw)
agg_year_ps = agg_year(df_ps_violations,nb_inspections_by_year_ps)

In [12]:
colors = ['#FFC72C', '#DA291C', '#f5d4b7', '#183E07','#008C15','#000000']
years = list(nb_inspections_by_year_mc.index)
for i in range(len(years)): years[i]=str(years[i])
years
data = {'Years' : years,
        'MCDONALDS'   : nb_inspections_by_year_mc.Count,
        'BURGERKING'  : nb_inspections_by_year_bk.Count,
        'KFC' : nb_inspections_by_year_kfc.Count,
        'HAROLDSCHICKEN'  : nb_inspections_by_year_hc.Count,
       'SUBWAY': nb_inspections_by_year_sw.Count,
       'POTBELLYSANDWICH': nb_inspections_by_year_ps.Count}

In [13]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r0 = p.line(x=years, y=agg_year_mc['inspection_fail_rate'],line_color=colors[0],line_width=3)
r1 = p.line(x=years, y=agg_year_bk['inspection_fail_rate'],line_color=colors[1],line_width=3)
legend = Legend(items=[
    ('MCDONALDS',[r0]),
    ('BURGERKING',[r1]),
]
, location="center")
p.add_layout(legend, 'right')
show(p)


In [14]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r2 = p.line(x=years, y=agg_year_kfc['inspection_fail_rate'],line_color=colors[2],line_width=3)
r3 = p.line(x=years, y=agg_year_hc['inspection_fail_rate'],line_color=colors[3],line_width=3)
legend = Legend(items=[
    ('KFC',[r2]),
    ('HAROLDSCHICKEN',[r3]),
]
, location="center")
p.add_layout(legend, 'right')
show(p)

In [15]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Inspection Fail Rate by year",
           toolbar_location=None)
r4 = p.line(x=years, y=agg_year_sw['inspection_fail_rate'],line_color=colors[4],line_width=3)
r5 = p.line(x=years, y=agg_year_ps['inspection_fail_rate'],line_color=colors[5],line_width=3)
legend = Legend(items=[
    ('SUBWAY',[r4]),
    ('POTBELLYSANDWICH',[r5]),
]
, location="center")
p.add_layout(legend, 'right')
show(p)

In [16]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Mean # of inspections by year",
           toolbar_location=None)
r6 = p.line(x=years, y=nb_inspections_by_year_mc.Count,line_color=colors[0],line_width=3)
r7 = p.line(x=years, y=nb_inspections_by_year_bk.Count,line_color=colors[1],line_width=3)
r8 = p.line(x=years, y=nb_inspections_by_year_kfc.Count,line_color=colors[2],line_width=3)
r9 = p.line(x=years, y=nb_inspections_by_year_hc.Count,line_color=colors[3],line_width=3)
r10 = p.line(x=years, y=nb_inspections_by_year_sw.Count,line_color=colors[4],line_width=3)
r11 = p.line(x=years, y=nb_inspections_by_year_ps.Count,line_color=colors[5],line_width=3)
legend = Legend(items=[
    ('MCDONALDS',[r6]),
    ('BURGERKING',[r7]),
    ('KFC',[r8]) ,
    ('HAROLDSCHICKEN',[r9]),
    ('SUBWAY',[r10]),
    ('POTBELLYSANDWICH',[r11]),]
, location="center")
p.add_layout(legend, 'right')
show(p)

In [21]:
p = figure(x_range=years, plot_height=300,plot_width=950, title="Fail rate by year",
           toolbar_location=None)
r0 = p.line(x=years, y=agg_year_mc['inspection_fail_rate'],line_color=colors[0],line_width=3)
r1 = p.line(x=years, y=agg_year_bk['inspection_fail_rate'],line_color=colors[1],line_width=3)
r2 = p.line(x=years, y=agg_year_kfc['inspection_fail_rate'],line_color=colors[2],line_width=3)
r3 = p.line(x=years, y=agg_year_hc['inspection_fail_rate'],line_color=colors[3],line_width=3)
r4 = p.line(x=years, y=agg_year_sw['inspection_fail_rate'],line_color=colors[4],line_width=3)
r5 = p.line(x=years, y=agg_year_ps['inspection_fail_rate'],line_color=colors[5],line_width=3)
legend = Legend(items=[
    ('MCDONALDS',[r0]),
    ('BURGERKING',[r1]),
    ('KFC',[r2]) ,
    ('HAROLDSCHICKEN',[r3]),
    ('SUBWAY',[r4]),
    ('POTBELLYSANDWICH',[r5]),]
, location="center")
p.add_layout(legend, 'right')
export_png(p, filename="Fail_rate.png")
show(p)

In [17]:
p = figure(x_range=years, plot_height=250, title="mean # of inspections by year",
           toolbar_location=None)
p.vbar_stack(['MCDONALDS','BURGERKING','KFC','HAROLDSCHICKEN','SUBWAY','POTBELLYSANDWICH'], x='Years', width=0.9, color=colors, source=data
            ,legend_label=['MCDONALDS','BURGERKING','KFC','HAROLDSCHICKEN','SUBWAY','POTBELLYSANDWICH'])
p.y_range.start = 0
p.y_range.range_padding = 1
p.xgrid.grid_line_color = None
p.axis.minor_tick_line_color = None
p.outline_line_color = None
p.legend.location = "top_center"
p.legend.orientation = "horizontal"
show(p)

The 'Sandwich' restaurant has more inspection by year that 'Burgers' and 'Chicken' restaurant. In addition, McDonalds has more inspections than Burger king but for the 'Chicken' restaurant, it's quite the same.